### TODO (project-level): 

- Generation of Cell names
- names for higher subclasses
- penotypic & Genotypic -- Check threshold
  
----------------------------------------------------

- Start building from individual neurons
- Naming scheme: Tasic 15, cluster [cluster_number]
- Function that changes names flexibly
- Templates & References
    - Bolser Lewis 
    - OntTerm triplesimple
    - nistd/core.py
    - curation.py = triplesExport
    
#### Code-level:
- Start using templates from NeuronLangExample
- Recapitulating the matrix with definitive phenotypes
- populate namespace with definitive phenotypesi

Definitive phenotype:
    - Cre line [NCBI Taxon] [Allen Brain Institute]
    - V1 
    - Layer of dissection [UBERON]

Inferred gene markers phenotype
    - Cluster ID [ILX]
    - Genes present [NCBIGene]
    - Genes Absent [NCBIGene]

#Tasic hiearchy
### For the Future:
- Continuous Phenotypes (gene counts, etc)
- Phenotype Inheritance in subclasses?

http://casestudies.brain-map.org/celltax

https://www.nature.com/articles/nn.4216

In [1]:
%%capture
#standard libraries
from pathlib import Path
from importlib import reload

#extended libraries
import requests
import rdflib
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

#tree stuff
import anytree
import anytree.util
from anytree.importer import DictImporter
from anytree.exporter import DotExporter

#custom imports
from neurondm import *
from neurondm.models.huang2017 import Genes
from pyontutils.namespaces import ilxtr as pred
from neurondm import phenotype_namespaces as phns
from pyontutils.closed_namespaces import rdf, rdfs, owl
#from nifstd.nifstd_tools.utils import ncbigenemapping
from utils import *

In [ ]:
def cellguard(addns=False):
    # unfortunately ipy.hooks['pre_run_code_hook'].add(__cellguard)
    # causes this to be called too frequently :/
    setLocalNames()
    setLocalContext()
    if addns:
        setLocalNames(phns.BBP)

In [2]:
df_clf = pd.read_csv('Data/cell_classification.csv')
df_cls_mtd = pd.read_csv('Data/cluster_metadata.csv')
df_cell_mtd = pd.read_csv('Data/cell_metadata.csv')

df_cre_mtd  = pd.read_excel('Data/tasic_crelines.xlsx')

dendro = pd.read_csv("Data/web/big_tree_final.csv", dtype = {'position':str})

In [ ]:
"""
Information about the cluster membership of each cell,
including whether the cell is a "core" (unambiguously assigned to a single cluster) 
or "transition" (shares membership between two clusters) cell, 
as well as its membership score (from 0-10) for each cluster (labeled f01 to f49). 
"""
df_clf.head()
#TODO: Only including unambiguous cells with f-values = 10 for one cluster only.

In [ ]:
"""
 Information about each data-driven cluster, including its label, 
 the corresponding label in Tasic et al. (Nat. Neuro, 2106), the primary cell class 
 membership, and marker genes (including genes with widespread expression in the cluster, 
 sparse expression in the cluster, and no expression in the cluster). 
"""
df_cls_mtd.head()

In [ ]:
"""
Information about each cell profiled, including its nomenclature, 
Cre line of origin, dissection, date of collection and sequencing, 
and read mapping statistics
"""
df_cell_mtd.head()

In [ ]:
df_clf['coretype'].value_counts()

In [ ]:
df_cell_mtd['layer_dissectoin'].value_counts()

In [ ]:
df_cell_mtd['cre'].value_counts()

In [3]:
df_clf.rename(index=str, columns={"Unnamed: 0":"cell_index"}, inplace = True)
df_clf = df_clf[['cell_index','coretype', 'primary', 'secondary']]

#Reorder and drop columns that are not interesting
df_cell_mtd = df_cell_mtd[['short_name','cre','major_class','sub_class',
                           'major_dissection', 'layer_dissectoin']]
df_cls_mtd = df_cls_mtd[['cluster_id','cluster_order','vignette_label',
                         'group','markers_present','markers_sparse',
                         'genes_absent','Tasic_et_al_2016_label']]

df_cls_mtd['cluster_id'] = df_cls_mtd['cluster_id'].astype(str)

#merge
df_types = df_cell_mtd.merge(df_clf, left_on='short_name', right_on='cell_index')
df_types = df_types[['short_name', 'coretype', 'primary', 'secondary',
                     'cre','major_dissection', 'layer_dissectoin']]

In [ ]:
#cre line conversion
df_cre_mtd.head()

In [ ]:
df_types.head()

In [ ]:
# metada query test
df_cre_mtd[df_cre_mtd['Abbreviation'] == df_types['cre'][1000]]

main dataframe: df_types

metadata: cre_mtd & cls_mtd

## Dendrogram Parsing

In [60]:
dendro.head()

,cluster,position
0,Cd34,000000
1,Ndnf Car4,0000010
2,Ndnf Cxcl14,0000011
3,Vip Gpc3,0000100
4,Vip Chat,0000101


In [61]:
tree_dict = {'label':'root'} #base dictionary

#parses binary node positions into a dictionary with tree structure
for label, bin_str in zip(dendro['cluster'], dendro['position']):
    parse_binary(tree_dict, bin_str, label)

In [62]:
importer = DictImporter()
tree = importer.import_(tree_dict)

for ind, node in enumerate(anytree.LevelOrderIter(tree)):
    node.pos = ind + 1 #node index starting at one
    if node.is_leaf:
        node.name = str(ind + 1) + " " + node.label
    else:
        node.name = str(ind + 1)
        
print(anytree.RenderTree(tree))
DotExporter(tree).to_picture("dendro.png")

#node.name for rendering
#node.pos & node.label used for parsing

AnyNode(label='root', name='1', pos=1)
├── AnyNode(name='2', pos=2)
│   ├── AnyNode(name='4', pos=4)
│   │   ├── AnyNode(name='8', pos=8)
│   │   │   ├── AnyNode(name='16', pos=16)
│   │   │   │   ├── AnyNode(name='28', pos=28)
│   │   │   │   │   ├── AnyNode(label='Cd34', name='42 Cd34', pos=42)
│   │   │   │   │   └── AnyNode(name='43', pos=43)
│   │   │   │   │       ├── AnyNode(label='Ndnf Car4', name='60 Ndnf Car4', pos=60)
│   │   │   │   │       └── AnyNode(label='Ndnf Cxcl14', name='61 Ndnf Cxcl14', pos=61)
│   │   │   │   └── AnyNode(name='29', pos=29)
│   │   │   │       ├── AnyNode(name='44', pos=44)
│   │   │   │       │   ├── AnyNode(label='Vip Gpc3', name='62 Vip Gpc3', pos=62)
│   │   │   │       │   └── AnyNode(label='Vip Chat', name='63 Vip Chat', pos=63)
│   │   │   │       └── AnyNode(name='45', pos=45)
│   │   │   │           ├── AnyNode(name='64', pos=64)
│   │   │   │           │   ├── AnyNode(label='Vip Cxcl14_Car4', name='80 Vip Cxcl14_Car4', pos=80)
│   │   │  

In [63]:
for leaf in tree.leaves: #TODO: phenotypes & deal with mismatching
    #edge cases for the last two endothelial cells
    if leaf.label  == 'Endo Tbc1d4':
        leaf.cluster_id = 'f48'
    elif leaf.label == 'Endo Myl9':
        leaf.cluster_id = 'f49'
    else:
        leaf.cluster_id = list(df_cls_mtd[df_cls_mtd['vignette_label'] == leaf.label]['cluster_id'])[0]
    
    #FIXME: label name matching somehow not working
    """
    genes_present = list(df_cls_mtd[df_cls_mtd['cluster_id'] == leaf.cluster_id]['markers_present'].str.split(","))
    genes_absent = list(df_cls_mtd[df_cls_mtd['cluster_id'] == leaf.cluster_id]['genes_absent'].str.split(","))
    if genes_present == np.nan:
        leaf.genes_present = set()
    if genes_absent == np.nan:
        leaf.genes_absent = set()
    else:
        leaf.genes_present = set(genes_present[0])
        #leaf.genes_absent = set(genes_absent)
    """

In [64]:
#gene and cluster columns converted from dendrogram structure
df_types['cluster'] = df_types.apply(cluster_converter, tree = tree, axis = 1)
df_types['markers_present'] = df_types.apply(gene_merge, df = df_cls_mtd, index = 'markers_present', axis = 1)
df_types['markers_absent'] = df_types.apply(gene_merge, df = df_cls_mtd, index = 'genes_absent', axis = 1)

In [ ]:
df_types[df_types['coretype'] == 'Transition']

In [ ]:
pd.value_counts(df_types[df_types['coretype'] == 'Transition']['cluster']).plot.bar()

### Phenotype Bagging

In [ ]:
#Tables should now be in final version
df_types.head()

In [ ]:
cre_df3 = pd.merge(df_cre_mtd, cre_ref,  how='inner', 
                  left_on='Public Repository Stock #',right_on = 'stock_number')

In [ ]:
cre_df3.drop_duplicates(inplace=True)
cre_df3.head()

In [ ]:
cre_df2 = pd.merge(df_cre_mtd, cre_ref,  how='inner', 
                  left_on='Driver Line',right_on = 'name')

In [ ]:
cre_df2

In [ ]:
cre_df = pd.merge(df_cre_mtd, cre_ref,  how='left', 
                  left_on=['Driver Line','Public Repository Stock #'], 
                  right_on = ['name','stock_number'])

In [ ]:
cre_df

In [25]:
response = requests.get('http://api.brain-map.org/api/v2/data/query.json?criteria='
                        'model::TransgenicLine,rma::options[num_rows$eqall]')
cre_ref = pd.DataFrame(response.json()['msg'])
cre_ref['stock_number'] = pd.to_numeric(cre_ref['stock_number'])

#merge both cre dataframes
cre_df1 = pd.merge(df_cre_mtd, cre_ref,  how='inner', 
                   left_on='Driver Line',right_on = 'name')

cre_df2 = pd.merge(df_cre_mtd.dropna(subset = ['Public Repository Stock #']), 
                   cre_ref.dropna(subset = ['stock_number']),  how='inner', 
                   left_on='Public Repository Stock #',
                   right_on = 'stock_number')

cre_df1.drop_duplicates(inplace = True)
cre_df2.drop_duplicates(inplace = True)
cre_df = pd.concat([cre_df1, cre_df2], axis = 0)
cre_df.drop_duplicates(inplace = True)
#get rid of redunant/useless columns


In [31]:
df_cell_mtd['cre'].value_counts()

Rbp4              173
Htr3a             123
Cux2              122
Sst               107
Scnn1a-Tg3         99
Pvalb              89
PvalbF-Gad2        87
Chrna2             84
Gad2               77
Ntsr1              77
Nos1               72
Vip                68
Calb2              64
Ntsr1 Neg          55
Rorb               54
Nr5a1              48
Slc17a6            47
PvalbD-Slc32a1     44
Scnn1a-Tg3 Neg     23
Rbp4 Neg           23
Ctgf               23
Nkx2-1             22
Scnn1a-Tg2         20
Ndnf               19
Scnn1a-Tg2 Neg     18
Cux2 Neg           15
Ndnf Neg           12
Tac2                8
Chat                6
Name: cre, dtype: int64

In [39]:
df_cre_mtd[df_cre_mtd['Abbreviation'] == 'Ndnf']

,#,Driver Line,Abbreviation,Originating Lab (Donating Investigator),Primary Reference,Generation Method,"Generation Method, more detail",Public Repository,Public Repository Stock #,Repository Strain Name,Data available through the Allen Institute Transgenic Portal
7,8.0,Ndnf-IRES2-dgCre,Ndnf,Allen Institute for Brain Science,This paper,Knock-in,IRES2,NaN,NaN,NaN,http://connectivity.brain-map.org/transgenic/s...


In [9]:
cre_df2 = pd.merge(cre_ref, df_cre_mtd,  how='inner', 
                   left_on='stock_number',
                   right_on = 'Public Repository Stock #')

In [ ]:
cre_df2 = pd.merge(df_cre_mtd, cre_ref,  how='inner', 
                   left_on='Public Repository Stock #',
                   right_on = 'stock_number')
cre_df2.drop_duplicates(inplace=True)

In [10]:
cre_df2

,description,id,name,originating_lab,stock_number,sub_image_annotation_id,transgenic_line_source_name,transgenic_line_type_code,transgenic_line_type_name,url_prefix,...,Driver Line,Abbreviation,Originating Lab (Donating Investigator),Primary Reference,Generation Method,"Generation Method, more detail",Public Repository,Public Repository Stock #,Repository Strain Name,Data available through the Allen Institute Transgenic Portal
0,"Cre expression is enriched in layers 2/3, and ...",306000233,Plxnd1-CreER,Z. Josh Huang,NaN,543091192.0,Other,D,driver,None,...,Ctgf-2A-dgCre,Ctgf,Allen Institute for Brain Science,This paper,Knock-in,T2A,NaN,NaN,NaN,http://connectivity.brain-map.org/transgenic/s...
1,"Cre expression is enriched in layers 2/3, and ...",306000233,Plxnd1-CreER,Z. Josh Huang,NaN,543091192.0,Other,D,driver,None,...,Ndnf-IRES2-dgCre,Ndnf,Allen Institute for Brain Science,This paper,Knock-in,IRES2,NaN,NaN,NaN,http://connectivity.brain-map.org/transgenic/s...
2,"Cre expression is enriched in layers 2/3, and ...",306000233,Plxnd1-CreER,Z. Josh Huang,NaN,543091192.0,Other,D,driver,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Cre expression is enriched in layers 2/3, and ...",306000233,Plxnd1-CreER,Z. Josh Huang,NaN,543091192.0,Other,D,driver,None,...,*Only used for isolation of cortical RNA for o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Scattered expression in cortical and subcortic...,617166869,Tacr1-T2A-Cre-neo,Allen Institute for Brain Science,NaN,623914828.0,AIBS,D,driver,None,...,Ctgf-2A-dgCre,Ctgf,Allen Institute for Brain Science,This paper,Knock-in,T2A,NaN,NaN,NaN,http://connectivity.brain-map.org/transgenic/s...
5,Scattered expression in cortical and subcortic...,617166869,Tacr1-T2A-Cre-neo,Allen Institute for Brain Science,NaN,623914828.0,AIBS,D,driver,None,...,Ndnf-IRES2-dgCre,Ndnf,Allen Institute for Brain Science,This paper,Knock-in,IRES2,NaN,NaN,NaN,http://connectivity.brain-map.org/transgenic/s...
6,Scattered expression in cortical and subcortic...,617166869,Tacr1-T2A-Cre-neo,Allen Institute for Brain Science,NaN,623914828.0,AIBS,D,driver,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Scattered expression in cortical and subcortic...,617166869,Tacr1-T2A-Cre-neo,Allen Institute for Brain Science,NaN,623914828.0,AIBS,D,driver,None,...,*Only used for isolation of cortical RNA for o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,None,561404352,PhiC31-neo,None,NaN,NaN,None,D,driver,None,...,Ctgf-2A-dgCre,Ctgf,Allen Institute for Brain Science,This paper,Knock-in,T2A,NaN,NaN,NaN,http://connectivity.brain-map.org/transgenic/s...
9,None,561404352,PhiC31-neo,None,NaN,NaN,None,D,driver,None,...,Ndnf-IRES2-dgCre,Ndnf,Allen Institute for Brain Science,This paper,Knock-in,IRES2,NaN,NaN,NaN,http://connectivity.brain-map.org/transgenic/s...


In [ ]:
cre_df = cre_df[['Abbreviation','name', 'id', 'stock_number',
                 'transgenic_line_source_name',
                 'transgenic_line_type_name',
                 'url_prefix', 'url_suffix','description']]

In [ ]:
cre_df2 = pd.merge(df_cre_mtd, cre_ref,  how='inner', 
                   left_on='Public Repository Stock #',
                   right_on = 'stock_number')

In [95]:
from pyontutils.namespaces import makePrefixes, ilxtr, definition
from pyontutils.closed_namespaces import rdf, rdfs, owl
from pyontutils.config import devconfig

class Tasic2015(Genes, phns.Species, 
                phns.Regions, phns.Layers):
    branch = devconfig.neurons_branch
    
    #TODO: Ambiguous, most likely post-clustering layers
    L6a = Phenotype(ilxtr.TasicL6a, ilxtr.hasSomaLocatedIn, 
                    label = "Tasic Layer VI - A", override = True)
    L6b = Phenotype(ilxtr.TasicL6b, ilxtr.hasSomaLocatedIn, 
                    label = "Tasic Layer VI - B", override = True)
    
    #Aggregate Layer Phenotypes
    with phns.Layers:
        upper = LogicalPhenotype(OR, L1, L2, L3)
        lower = LogicalPhenotype(OR, L4, L5, L6)
        All = LogicalPhenotype(OR, upper, lower)
    
    #Tasic Hiearchical Cluster Position (1-indexed, breadth first)
    cmp  = pred.hasComputedMolecularPhenotype

class TasicBagger:
    #from Allen Cell Types
    prefixes = {**{'JAX': 'http://jaxmice.jax.org/strain/',
                   'MMRRC': 'http://www.mmrrc.org/catalog/getSDS.jsp?mmrrc_id=',
                   'AllenTL': 'http://api.brain-map.org/api/v2/data/TransgenicLine/'},
                **makePrefixes('definition', 'ilxtr', 'owl')}

    def __init__(self, data = None, **metadata):
        """
        Initializes a Tasic Neuron Bagger Object
        
        data: Pandas DataFrame
        **metadata: any relevant metadata DataFrames
        """
        self.data = data
        self.ns = {k:rdflib.Namespace(v) for k, v in self.prefixes.items()}
        if metadata:
            for key, dataframe in metadata.items():
                setattr(self, key, dataframe)
            
    @staticmethod
    def layer_parse(key):
        """method that parses the layer labels for Tasic 2015 cells.
        returns the layer phenotype.
        
        key: str, layer label
        """
        if key == 'L2/3':
            layer_phn = L23     
        else:
            layer_phn = Tasic2015[key]
        
        return layer_phn
    
    @staticmethod
    def gene_parse(gene_set, mode = 'present'):
        """method that parses a set of gene markers and returns
        a list of phenotypes.
        
        gene_set: set, set of gene markers in dtype str
        mode: str, "present" or "absent" markers.
        """
        gene_phns = []
        undef = set()
        f = Phenotype if mode == 'present' else NegPhenotype
        for gene in gene_set:
            if gene in Genes.__dict__:
                gene_phns.append(f(Genes[gene]))
            else:
                undef.add(gene)
        if len(undef) > 0: #FIXME: some genes cannot be mapped
            mappings, to_add, errors = ncbigenemapping(undef, 
                                        return_errors = True)
            for gene_name, iri in mappings.items():
                gene_phns.append(f(iri, ilxtr.hasExpressionPhenotype,
                                   label = gene_name, override = True))
        if len(gene_phns) != len(gene_set):
            print(errors)
        return gene_phns
    
    @staticmethod
    def cluster_parse(pos):
        """Tasic Computed Gene-based hiearchical cluster
        pos: int
        """
        cluster_phn = Phenotype("ilxtr:cluster" + str(pos), cmp, 
                                label = str(pos))
        return cluster_phn
    
    
    def transgenic_parse(self, row):
        phenotypes = []
        pred = 'ilxtr:hasDriverExpressionPhenotype'
        cre = self.cre_metadata[self.cre_metadata['Abbreviation'] == row['cre']]
        cre_phn = Phenotype('PR:000013502', pred.hasExpressionPhenotype)
        ##WHERE R THE IRIs ??
        #for tl in cell_line['donor']['transgenic_lines']:
            #prefix = self.cre_metadata['Driver Line']
            #suffix = self.cre_metadata['Public Repository Stock #'] if tl['stock_number'] else str(tl['id'])
            #line_names = []
            #if prefix and suffix and prefix in ['AIBS', 'MMRRC', 'JAX']:
                #if prefix == 'AIBS':
                   # prefix = 'AllenTL'
               # iri = self.ns[prefix][suffix]
               # phenotypes.append(Phenotype(iri, pred))
        return cre_phn
    
    def build_transgenic_lines(self):
        triples = []
        for ind, tl in self.cre.iterrows():
            _id = tl['stock_number'] if tl['stock_number'] else tl['id']
            prefix = tl['transgenic_line_source_name']
            line_type = tl['transgenic_line_type_name']
            if prefix not in ['JAX', 'MMRRC', 'AIBS']:
                print('WARNING:', 'unknown prefix')
                continue
            elif prefix == 'AIBS':
                prefix = 'AllenTL'

            _class = self.ns[prefix][str(_id)]
            triples.append((_class, rdf.type, owl.Class))
            triples.append((_class, rdfs.label, rdflib.Literal(tl['name'])))
            triples.append((_class, definition, rdflib.Literal(tl['description'])))
            triples.append((_class, rdfs.subClassOf, ilxtr.transgenicLine))
            triples.append((_class, ilxtr.hasTransgenicType, ilxtr[line_type + 'Line']))

        # TODO aspects.ttl?
        transgenic_lines = simpleOnt(filename='tasic-transgenic-lines',
                                     path='ttl/generated/',
                                     prefixes=self.prefixes,
                                     triples=triples,
                                     comment='Tasic transgenic lines for cell types',
                                     branch=self.branch)

        transgenic_lines._graph.write()
        
        #return transgenic_lines   
    
    @property
    def bags(self):
        with Tasic2015:
            #Every neuron sampled in this paper is from V1
            with Neuron(phns.Species.Mouse, phns.Regions.V1) as context:
                for row in self.data.itertuples():
                    label = str(row.short_name) #change this
                    #cluster_phn = Phenotype(str(row.cluster), cmp)
                    #layer_phn = TasicBagger.layer_parse(row.layer_dissectoin)

                    present_phns = TasicBagger.gene_parse(row.markers_present, mode = 'present')
                    absent_phns  = TasicBagger.gene_parse(row.markers_absent, mode = 'absent')
                    markers_phns = present_phns + absent_phns

                    #phenotypes = [layer_phn, cre_phn, cluster_phn] + markers_phns
                    phenotypes = markers_phns
                    yield label, phenotypes
          
    
class TasicNeuron(NeuronEBM):
    owlClass = ilxtr.NeuronTasic2015
    shortname = 'Tasic2015'

In [96]:
metadata = {"cre":cre_df}

def main():
    ttl_test_path = '/var/host/media/removable/SD Card/Neuron/Tasic/ttl_export'
    config = Config("tasic-2015", ttl_export_dir=Path(ttl_test_path))
    tb = TasicBagger(data = df_types, **metadata)
    ind = 0
    for label, bag in tb.bags:
        if ind == 2:
            break
        TasicNeuron(*bag, label = label, override=True)
        ind += 1
    config.write()
    config.write_python()

In [104]:
config.write_python()

TypeError: <module '__main__'> is a built-in class

In [99]:
ttl_test_path = '/var/host/media/removable/SD Card/Neuron/Tasic/ttl_export'
config = Config("tasic-2015", ttl_export_dir=Path(ttl_test_path))
tb = TasicBagger(data = df_types, **metadata)
ind = 0
for label, bag in tb.bags:
    if ind == 1:
        break
    TasicNeuron(*bag, label = label, override=True)
    ind += 1
config.write()
config.write_python()

[('22004', 'Tpm2', '10090'), ('57266', 'Cxcl14', '10090'), ('209378', 'Itih5', '10090'), ('12862', 'Cox6a2', '10090')]
[('20660', 'Sorl1', '10090'), ('12351', 'Car4', '10090')]
[('12351', 'Car4', '10090'), ('21334', 'Tac2', '10090')]
[('18167', 'Npy2r', '10090'), ('242259', 'Slc44a5', '10090')]


TypeError: <module '__main__'> is a built-in class

In [97]:
from neurondm.sheets import Sheet
from neurondm import OntId, OntTerm, Config, NeuronEBM, Neuron
from pyontutils.utils import byCol, relative_path
from pyontutils.namespaces import ilxtr
from pyontutils.closed_namespaces import rdfs

main()

[('22004', 'Tpm2', '10090'), ('57266', 'Cxcl14', '10090'), ('209378', 'Itih5', '10090'), ('12862', 'Cox6a2', '10090')]
[('20660', 'Sorl1', '10090'), ('12351', 'Car4', '10090')]
[('12351', 'Car4', '10090'), ('21334', 'Tac2', '10090')]
[('18167', 'Npy2r', '10090'), ('242259', 'Slc44a5', '10090')]
[('73750', 'Whrn', '10090'), ('18573', 'Pde1a', '10090'), ('16323', 'Inhba', '10090'), ('13602', 'Sparcl1', '10090'), ('109593', 'Lmo3', '10090'), ('225998', 'Rorb', '10090')]
[('320265', 'Tafa1', '10090'), ('17311', 'Kitl', '10090'), ('19011', 'Endou', '10090')]


TypeError: <module '__main__'> is a built-in class

In [ ]:
metadata = {"cre":cre_df}
tb = TasicBagger(data = df_types, **metadata)
tb.build_transgenic_lines()

### Test Code

In [51]:
with Tasic2015:
    print(Mouse)

Phenotype('NCBITaxon:10090',
          'ilxtr:hasInstanceInSpecies',
          label='Mus musculus')


In [ ]:
from pyontutils.core import simpleOnt
simpleOnt()

In [ ]:
#Phenotype("ilxtr:cluster6", "ilxtr:hasComputedMolecularPhenotype", label = "Tasic", check = False)

In [ ]:
from pyontutils.utils import byCol, relative_path
from pyontutils.namespaces import ilxtr
from pyontutils.closed_namespaces import rdfs

In [ ]:
with Neuron(phns.Species.Mouse, phns.Regions.V1) as context:
    print(context)
    n11 = Neuron(phns.Layers.L1)
    print(n11)